In [ ]:
!pip install gtts
!pip install moviepy

In [ ]:
import subprocess
from gtts import gTTS
from moviepy.editor import VideoClip, ImageSequenceClip,ImageClip
from moviepy.editor import VideoFileClip, concatenate_videoclips
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip
import cv2
import os
import imageio
import subprocess
from moviepy.video.tools.subtitles import SubtitlesClip
from moviepy.editor import VideoFileClip
from google.colab import drive

drive.mount('/content/drive')

num_images_per_prompt=5

def create_video(image_folder, video_name, fps, i, num_images=num_images_per_prompt):
    image_files = [img for img in os.listdir(image_folder) if any(img.endswith(f"_{i}_{j}.png") for j in range(num_images))]
    image_files.sort()

    video_path = os.path.join(image_folder, video_name)
    writer = imageio.get_writer(video_path, fps=fps)

    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        frame_bgr = cv2.imread(image_path)
        frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        writer.append_data(frame_rgb)

    writer.close()



v=[]

file_path = '/content/drive/MyDrive/prompts/txtfile.txt'

prompts = []
with open(file_path, "r") as file:
    for line in file:
        prompts.append(line.strip())

for i in range(len(prompts)):
    image_folder = "/content/drive/MyDrive/prompts"
    video_name = f"video_{i}.mp4"
    create_video(image_folder, video_name,1,i)


    text_to_speak = prompts[i]
    language = 'en'
    tts = gTTS(text=text_to_speak, lang=language, slow=False)
    audio_file_path = f"/content/drive/MyDrive/prompts/audio_{i}.mp3"
    tts.save(audio_file_path)

    video_path = f"/content/drive/MyDrive/prompts/video_{i}.mp4"
    audio_path = f"/content/drive/MyDrive/prompts/audio_{i}.mp3"
    output_path = f"/content/drive/MyDrive/prompts/video_with_audio_{i}.mp4"
    video_clip = VideoFileClip(video_path)
    audio_clip = AudioFileClip(audio_path)
    video_clip = video_clip.set_audio(audio_clip)

    video_clip.write_videofile(output_path, codec="libx264", audio_codec="aac", fps=video_clip.fps)
    # Input and output file paths
    input_video_path = output_path
    output_video_path = f"/content/drive/MyDrive/prompts/video_with_sub_{i}.mp4"  # Specify the complete file path
    subtitles_text = prompts[i]

    # Generate subtitles file dynamically
    with open("/content/subtitles.srt", "w") as f:
        f.write("1\n00:00:00,000 --> 00:00:10,000\n" + subtitles_text)

    # Use subprocess to run ffmpeg command
    ffmpeg_command = f"ffmpeg -i '{input_video_path}' -vf 'subtitles=/content/subtitles.srt' -c:a copy '{output_video_path}'"
    subprocess.run(ffmpeg_command, shell=True)
    #video_clip.write_videofile(output_path, codec="libx264", audio_codec="aac", fps=video_clip.fps)


In [ ]:
# Assuming 'prompts' is a list of video prompts

# Paths to the video files
video_paths = [f"/content/drive/MyDrive/prompts/video_with_sub_{i}.mp4" for i in range(len(prompts))]

# Output video file path
output_path = "/content/drive/MyDrive/prompts/output_concatenated_video_new.mp4"

# Load video clips
clips = [VideoFileClip(video_path) for video_path in video_paths]

# Concatenate the video clips
final_clip = concatenate_videoclips(clips, method="compose")

# Write the final result to a new video file with explicit fps
final_clip.write_videofile(output_path, codec="libx264", fps=30)  # Adjust the fps value as needed


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Assuming your video is in the content folder
!cp //content/drive/MyDrive/prompts/output_concatenated_video_new.mp4 /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
